In [ ]:
# Neccessary statements to run colab from Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Neccessary statements
!mkdir mywork
!mkdir mydata
!cp -r /content/drive/MyDrive/FinalProjectLinearProgramming///common_data_indicators_american_community_survey_tracts_2017_2021.csv /content/mydata
!cp -r /content/drive/MyDrive/FinalProjectLinearProgramming///coloradoCarshareLocations1.csv /content/mydata

mkdir: cannot create directory ‘mywork’: File exists
mkdir: cannot create directory ‘mydata’: File exists


In [ ]:
# install pyomo and gurobipy
!pip install pyomo
!pip install gurobipy

In [ ]:
# Import neccessary packages
import geopandas as gpd
import pandas as pd
from scipy.optimize import linprog
import pyomo
import numpy as np
import gurobipy
from numpy.core.multiarray import empty

In [ ]:
# imort needed csv's
census_tract = pd.read_csv("./mydata/common_data_indicators_american_community_survey_tracts_2017_2021.csv")
CCS = pd.read_csv("./mydata/coloradoCarshareLocations1.csv", encoding='latin1')

# Create dictionary
car_count = {}

# Create a column of 0's
census_tract['carCount'] = np.zeros(176)

# Fill car_count dictionary with car counts
for i in np.unique(CCS['TRACT_NAME'].to_numpy()):
  index = CCS['TRACT_NAME'] == i

  num = CCS['car type'][index].shape[0]

  car_count[i] = num

car_count

{'Census Tract 11.02': 1,
 'Census Tract 16.03': 1,
 'Census Tract 17.04': 5,
 'Census Tract 17.05': 2,
 'Census Tract 17.06': 3,
 'Census Tract 19.01': 2,
 'Census Tract 21.02': 3,
 'Census Tract 24.04': 1,
 'Census Tract 24.05': 1,
 'Census Tract 26.03': 2,
 'Census Tract 27.05': 1,
 'Census Tract 27.07': 1,
 'Census Tract 27.09': 2,
 'Census Tract 29.01': 1,
 'Census Tract 31.02': 1,
 'Census Tract 32.02': 1}

In [ ]:
# Create a data frame from the dictionary
car_count_df = pd.DataFrame.from_dict(car_count, orient='index').reset_index()
car_count_df.columns = ['TRACT_NAME', 'count']

# Create a merged data frame
merged_df = pd.merge(census_tract, car_count_df, on='TRACT_NAME', how='left')

# fill the census_tract carCount column with merged data frame
census_tract['carCount'] = merged_df['count'].fillna(0)

In [ ]:
# Create a dictionary to hold tracts with cars already in it
T = {}

# Fill dictionary with CarCount
for i, id in enumerate(census_tract['TRACT_NAME']):

  T[id] = census_tract['carCount'].iloc[i]

# Create a dictionary to hold population of each tract
P = {}

# Fill dictionary with PopulationDensity
for i, id in enumerate(census_tract['TRACT_NAME']):

  P[id] = census_tract['Nmbr_PopulationDensity'].iloc[i]


# Create a dictionary to hold weights - can be changed below
w = {}

# Fill dictionary with 1's (Can be changed later to add different weights)
for id in census_tract['TRACT_NAME']:

  w[id] = 1


# Make ratios of density to carCount to get the max density
ratios = {}

for id, car_count , pop in zip(census_tract['TRACT_NAME'], census_tract['carCount'].to_numpy(), census_tract['Nmbr_PopulationDensity'].to_numpy()):

      ratios[id] = car_count / pop

ratios

{'Census Tract 1.02': 0.0,
 'Census Tract 10': 0.0,
 'Census Tract 11.01': 0.0,
 'Census Tract 11.02': 9.993012635739776e-05,
 'Census Tract 119.02': 0.0,
 'Census Tract 119.03': 0.0,
 'Census Tract 120.01': 0.0,
 'Census Tract 120.10': 0.0,
 'Census Tract 120.15': 0.0,
 'Census Tract 120.16': 0.0,
 'Census Tract 13.01': 0.0,
 'Census Tract 13.02': 0.0,
 'Census Tract 14.01': 0.0,
 'Census Tract 14.02': 0.0,
 'Census Tract 14.03': 0.0,
 'Census Tract 15': 0.0,
 'Census Tract 153': 0.0,
 'Census Tract 154': 0.0,
 'Census Tract 155': 0.0,
 'Census Tract 156': 0.0,
 'Census Tract 157': 0.0,
 'Census Tract 158': 0.0,
 'Census Tract 16.01': 0.0,
 'Census Tract 16.02': 0.0,
 'Census Tract 16.03': 6.302491402582403e-05,
 'Census Tract 17.03': 0.0,
 'Census Tract 17.04': 0.0003939593242938573,
 'Census Tract 17.05': 9.306465681690038e-05,
 'Census Tract 17.06': 0.0002524912824586286,
 'Census Tract 17.07': 0.0,
 'Census Tract 18': 0.0,
 'Census Tract 19.01': 0.0005349372293515011,
 'Census Tra

In [ ]:
# Create a list
numbers = []

# Fill list to get the max of rations
for key in list(ratios):
  numbers.append(ratios[key])

M = max(numbers)

# Create a hard coded M, can be changed with
M

0.001138084158369045

In [ ]:
# Create AMPL shell
from IPython.terminal.interactiveshell import default
# Start pyomo environment
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# ser R
model.R = Set(initialize=census_tract['TRACT_NAME'])


# Parameters
model.C = Param(initialize = 27)
model.m = Param(initialize = M)
model.T = Param(model.R, initialize = T)
model.P = Param(model.R,initialize = P)
model.w = Param(model.R, initialize = w )

# Variables
model.x = Var(model.R, within=NonNegativeIntegers)

# Check sizes of variables to make sure are all the same
print("Size of P:", len(model.P))
print("Size of T:", len(model.T))
print("Size of x:", len(model.x))
print("Size of R:", len(model.R))
print(w)

Size of P: 176
Size of T: 176
Size of x: 176
Size of R: 176
{'Census Tract 1.02': 26.47584973, 'Census Tract 10': 45.93363696, 'Census Tract 11.01': 34.15977961, 'Census Tract 11.02': 29.25551814, 'Census Tract 119.02': 39.07407407, 'Census Tract 119.03': 22.80513919, 'Census Tract 120.01': 26.62662663, 'Census Tract 120.10': 22.98897509, 'Census Tract 120.15': 36.27218935, 'Census Tract 120.16': 31.04181432, 'Census Tract 13.01': 46.94214876, 'Census Tract 13.02': 38.4083045, 'Census Tract 14.01': 39.25959781, 'Census Tract 14.02': 42.3028786, 'Census Tract 14.03': 43.19809069, 'Census Tract 15': 38.56304985, 'Census Tract 153': 46.55647383, 'Census Tract 154': 31.41210375, 'Census Tract 155': 50.4743833, 'Census Tract 156': 19.33998465, 'Census Tract 157': 27.82118056, 'Census Tract 158': 47.98154556, 'Census Tract 16.01': 21.45922747, 'Census Tract 16.02': 23.40597256, 'Census Tract 16.03': 49.51395422, 'Census Tract 17.03': 36.15, 'Census Tract 17.04': 39.68816442, 'Census Tract 17

In [ ]:
# Objective function
max_cars_added = 5

model.obj = Objective(expr=sum(model.w[i]*model.P[i] * (model.T[i] + model.x[i]) for i in census_tract['TRACT_NAME'].to_list()), sense=maximize)

# Constraints
model.constraints = ConstraintList()
for i in model.R:
    model.constraints.add(model.m * (model.T[i] + model.x[i]) <= model.P[i])

model.constraints.add(sum(model.x[i] for i in model.R) <= model.C)
for i in model.R:
  model.constraints.add(model.x[i] <= max_cars_added)


# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model)

# Display the results
print("Objective Value:", model.obj())
print("Number of cars added to each census tract:")
for i in model.R:
  print(f"Census Tract {i}: {model.x[i]()}")

Objective Value: 47723705.59691025
Number of cars added to each census tract:
Census Tract Census Tract 1.02: 0.0
Census Tract Census Tract 10: 0.0
Census Tract Census Tract 11.01: 0.0
Census Tract Census Tract 11.02: 0.0
Census Tract Census Tract 119.02: 0.0
Census Tract Census Tract 119.03: 0.0
Census Tract Census Tract 120.01: 0.0
Census Tract Census Tract 120.10: 0.0
Census Tract Census Tract 120.15: 0.0
Census Tract Census Tract 120.16: 0.0
Census Tract Census Tract 13.01: 0.0
Census Tract Census Tract 13.02: 0.0
Census Tract Census Tract 14.01: 0.0
Census Tract Census Tract 14.02: 0.0
Census Tract Census Tract 14.03: 0.0
Census Tract Census Tract 15: 0.0
Census Tract Census Tract 153: 0.0
Census Tract Census Tract 154: 0.0
Census Tract Census Tract 155: 0.0
Census Tract Census Tract 156: 0.0
Census Tract Census Tract 157: 0.0
Census Tract Census Tract 158: 0.0
Census Tract Census Tract 16.01: 0.0
Census Tract Census Tract 16.02: 0.0
Census Tract Census Tract 16.03: 0.0
Census Tra

In [ ]:
# Create a dictionary to hold tract's with added cars
tract_added = {}

# Fill dictionary
for i in model.R:

  if model.x[i]() != 0:

    tract_added[i] = model.x[i]()

# Display results
tract_added

{'Census Tract 17.03': 5.0,
 'Census Tract 26.04': 5.0,
 'Census Tract 27.05': 5.0,
 'Census Tract 27.09': 5.0,
 'Census Tract 32.04': 2.0,
 'Census Tract 70.90': 5.0}

In [ ]:
# Create dictionary to hold tract;s with added and original cars
tract_new = {}

for i in model.R:

  if (model.T[i] + model.x[i]()) > 0:

    tract_new[i] = model.x[i]() + model.T[i]

tract_new

{'Census Tract 11.02': 1.0,
 'Census Tract 16.03': 1.0,
 'Census Tract 17.03': 5.0,
 'Census Tract 17.04': 5.0,
 'Census Tract 17.05': 2.0,
 'Census Tract 17.06': 3.0,
 'Census Tract 19.01': 2.0,
 'Census Tract 21.02': 3.0,
 'Census Tract 24.04': 1.0,
 'Census Tract 24.05': 1.0,
 'Census Tract 26.03': 2.0,
 'Census Tract 26.04': 5.0,
 'Census Tract 27.05': 6.0,
 'Census Tract 27.07': 1.0,
 'Census Tract 27.09': 7.0,
 'Census Tract 29.01': 1.0,
 'Census Tract 31.02': 1.0,
 'Census Tract 32.02': 1.0,
 'Census Tract 32.04': 2.0,
 'Census Tract 70.90': 5.0}

In [ ]:
# Pct HouseholdsNoVehAccess
w = {}

for i, id in enumerate(census_tract['TRACT_NAME']):

  w[id] = census_tract['Pct_HouseholdsNoVehAccess'].iloc[i]

In [ ]:
# Pct PopulationInPoverty
w = {}

for i, id in enumerate(census_tract['TRACT_NAME']):

  w[id] = census_tract['Pct_PopulationInPoverty'].iloc[i]

In [ ]:
# Pct Renters
w = {}

for i, id in enumerate(census_tract['TRACT_NAME']):

  w[id] = census_tract['Pct_OccupiedUnitsWithRenters'].iloc[i]

In [ ]:
# Pct cost burdened
w = {}

for i, id in enumerate(census_tract['TRACT_NAME']):

  w[id] = census_tract['Pct_HouseholdsCostBurdened'].iloc[i]